In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import io
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time

from collections import namedtuple
from IPython.display import clear_output
from tqdm.notebook import tqdm
from numpy.typing import NDArray

from scipy.optimize import minimize
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from qiskit.circuit import Parameter
from qiskit.circuit.library import RXGate, RYGate, RZGate, CXGate, CZGate, IGate
from qiskit_aer import AerSimulator

In [4]:
from qml.model.gate import get_gateset
from qml.model.unit import Unit, EmbedUnit

In [72]:
class Model:

    def __init__(
            self,
            num_qubit: int,
            num_output: int,
            input_units: Unit,
            fixed_units: list[Unit],
            trainable_units: list[Unit] = None,
            shots: int = 100,
            sim = None
    ):
        if not hasattr(fixed_units, "__len__"):
            fixed_units = [fixed_units]
        if trainable_units is not None and not hasattr(trainable_units, "__len__"):
            trainable_units = [trainable_units]
        self.nq = num_qubit
        self.nc = num_output
        self._input_units = input_units
        self._fixed_units = fixed_units
        self._trainable_units = trainable_units
        self._shots = shots
        self._sim = sim if sim is not None else AerSimulator()
        
    def forward(self, x, params=None, shots=None) -> float:
        if params is None:
            params = [unit.values for unit in self._trainable_units]
        if shots is None:
            shots = self._shots
        feed_dict = self._input_units.feed_dict(x)
        for unit in self._fixed_units:
            feed_dict[unit] |= unit.feed_dict()
        for unit, param in zip(self._trainable_units, params):
            feed_dict |= unit.feed_dict(param)
        
        bc = qc.assign_parameters(feed_dict)
        job = transpile(bc, self._sim)
        res = self._sim.run(job, shots=shots).result().get_counts()
        pre = res.get("0", 0) - res.get("1", 0)
        return pre / shots

    def _apply(self):
        qc = QuantumCircuit(self.nq, self.nc)

        self._input_units.apply_to_qc(qc)
        [
            fixed_unit.apply_to_qc(qc)
            for fixed_unit in self._fixed_units
        ]
        [
            trainable_unit.apply_to_qc(qc)
            for trainable_unit in self._trainable_units
        ]

        qc.measure(0, 0)
        return qc
    
    @property
    def input_units(self):
        return self._input_units
    
    @property
    def fixed_units(self):
        return self._fixed_units
    
    @property
    def trainable_units(self):
        return self._trainable_units
    
    @trainable_units.setter
    def trainable_units(self, units):
        self._trainable_units.append(units)
    
    def  fix_trainable_unit(self):
        [
            self._fixed_units.append(trainable_unit)
            for trainable_unit in self._trainable_units
        ]
        self._trainable_units = []
        
    @property
    def shots(self):
        return self._shots
    
    @shots.setter
    def shots(self, value):
        self._shots = value
        
    @property
    def trainable_parameters(self):
        return [
            unit.parameters for unit in self._trainable_units
        ]
    
    def update_parameters(self, new_parameters):
        for unit, param in zip(self.trainable_units, new_parameters):
            unit.values = param

In [76]:
nq = 2
ng = 3
gateset = get_gateset(nq)
inunit = EmbedUnit.generate_ry_arcsin_embed_unit(
    "inunit", nq, 1, gateset
)
fxunit = []
trunit = Unit.generate_random_unit(
    "trunit_1", nq, ng, gateset
)
model = Model(nq, 1, inunit, fxunit, trunit)
qc = model._apply()
qc.draw("mpl")

model.forward(1)

-0.04

In [81]:
def calc_gradients(model, x, shots=100):
    trainable_params = model.trainable_parameters
    tp_shapes = [len(tp) for tp in trainable_params]
    tp_shapes.insert(0, 0)
    tp_shape_idxs = np.cumsum(tp_shapes)
    
    trainable_params = np.hstack(trainable_params)
    demi_pi = np.pi / 2
    deux_pi = np.pi * 2
    
    def deflatten(flattened):
        return [
            flattened[idx_de:idx_to]
            for idx_de, idx_to
            in zip(tp_shape_idxs[:-1], tp_shape_idxs[1:])
        ]
    
    def calc_gradient_idx(idx):
        shifted_pos = trainable_params.copy()
        shifted_neg = trainable_params.copy()
        shifted_pos[idx] = (trainable_params[idx] + demi_pi) % deux_pi
        shifted_neg[idx] = (trainable_params[idx] - demi_pi) % demi_pi
        
        predict_pos = model.forward(
            x,
            params=deflatten(shifted_pos),
            shots=shots
        )
        predict_neg = model.forward(
            x,
            params=deflatten(shifted_neg),
            shots=shots
        )
        grad = (predict_pos - predict_neg) / 2
        return grad
    
    grads = np.asarray([
        calc_gradient_idx(idx)
        for idx in range(len(trainable_params))
    ])
    
    return deflatten(grads)
    

lr = 1e-2
grads = calc_gradients(model, 0)
new_params = [
    unit.values - grad * lr
    for unit, grad in zip(model.trainable_units, grads)
]
model.update_parameters(new_params)

In [82]:
model.trainable_parameters

[array([0.0138, 0.0], dtype=object)]